# Notes
- 9010 rated all arousal as 1
- 9028 rated all arousal as 2 during cortisol runs
- 9029's placebo run's memory is missing; experimenter error - they were shown the wrong stimuli at encoding, so their memory wasn't actually tested at retrieval

exlude 9029 for memory analysis
check 9010 and 9028 for arousal related analysis

In [1]:
import pandas as pd
import numpy as np
from scipy import signal as sg
import scipy.io

# read in time series

In [2]:
def create_timeseries(data_dir,sub,cond,stim):
    """
    compress timeseries of nodes into NxM matrix, where M = number of nodes, and N = number of timepoints
    Returns M * N matrix
    author:Flory Huang 
    Date: 08/29/2022
    """
    #size: 377 * scan length
    timeseries = []
    #read timeseries per nodes
    for i in range(377):
        my_file = open(data_dir+sub+'_BPfiltered_mean_timeseries/'+cond+'_enc_'+stim+'_node'+str(i+1)+'_meants.txt', 'r')
        data = my_file.read()
        ls = data.split('  \n')
        complete_ls = [i for i in ls if i!='' ]
        timeseries.append(complete_ls)
    return np.array(timeseries)

# phase coherence-based connectivity

In [3]:
def cosine_similarity(timeseries):
    """
    Function to calculate similarity between timeseries as a
    function of the angle of the complex representation
    Takes NxM matrix, where M = number of timeseries, and 
    N = number of timepoints
    Returns a matrix of size N x M x M
    """
    
    n_ts=timeseries.shape[1]
    n_tp=timeseries.shape[0]
    hilt = sg.hilbert(timeseries,axis=0)
    angles = np.angle(hilt)

    pw_diff=np.array([angles[v,:] - a for v in range(0,n_tp) for a in angles[v,:]])
    pw_diff=np.reshape(pw_diff,[n_tp,n_ts,n_ts])

    cos_sim=np.cos(pw_diff)

    return cos_sim

In [4]:
def create_ipmats_perSubj(data_dir,sub_ls,cond_ls,stim_ls,folder,timewindow=10,sliding=0,):
    #compress timeseries of nodes into NxM matrix, where M = number of nodes, and N = number of timepoints
    #sub_ls=["9001","9002","9003","9004","9005","9006","9007","9008","9010","9011","9013","9016"]
    #loop through all conditions
    sub_id=[]
    for sub in sub_ls:
        df_all=pd.read_csv('../data/behavior/mem_allsub_alltrial_pill.csv')
        for cond in cond_ls:
            for stim in stim_ls:
                print(sub,cond,stim)
                ipmats = np.array([])
                #create timeseries matrix for each run
                #size: 377 * scan length
                timeseries = create_timeseries(data_dir,sub,cond,stim)
                #size: 377*trial_length
                df=df_all[(df_all['Subject']==int(sub))&(df_all['Pill']==cond.title())&(df_all['Stim']==stim)]

                df=df.sort_values('Enc_Trial').reset_index(drop=True)
                for t in df['Enc_Trial'].values:
                    #select timewindow for computing connectivity per trial
                    trial_start=df[df['Enc_Trial']==t]['StimOn'].values[0]+sliding
                    current=timeseries[:,trial_start:trial_start+timewindow]
                    #skip trials at end that are not fully scanned.
                    if(np.shape(current)[1]<5):
                        continue;
                    #size: trail_length * (377)
                    timeseries_t = np.transpose(current)
                    #calculate connectivity matrix
                    cos_sim=cosine_similarity(timeseries_t)
                    
                    #take lower triangle of each connectivity matrix
                    # halfed_cos_sim = np.array([m[np.triu_indices(m.shape[0], k = 1)] for m in cos_sim])
                    
                    #calculate average connectivity across 10 sec
                    avg_cos_sim=np.mean(cos_sim,axis=0)
                    if (avg_cos_sim==-np.inf).sum()>0:
                        print(sub,con,stim)
                    #append all connectivity array
                    if ipmats.shape[0]<1:
                        ipmats=np.array([avg_cos_sim])
                    else:
                        ipmats=np.append(ipmats,[avg_cos_sim],axis=0)
                print(np.shape(ipmats))
                
                file_path = folder+sub+"_"+cond+"_"+stim+"_dynamic_connectome.mat"
                scipy.io.savemat(file_path, {'data': ipmats})

In [8]:
#sub_ls=["9001","9002","9003","9004","9005","9006","9007","9008","9010","9011","9013","9016","9018","9022","9027","9028","9030","9031","9033","9036","9037" ,"9038","9039","9040","9041"]
#rec_ls,skip_trial=trial_info(sub_ls,cond_ls,stim_ls)
sub_ls=["9001","9002","9003","9004","9005","9006","9007","9008","9010","9011","9013","9016","9018","9021","9022","9027","9028","9029","9030","9031","9033","9036","9037" ,"9038","9039","9040","9041"]

cond_ls=['cortisol','placebo']
stim_ls=['alc','tool']

data_dir='../data/BPfiltered_mean_timeseries/'
result_folder="../results/dynamic_connectivity/per_subj/";            

print(len(sub_ls))
create_ipmats_perSubj(data_dir,sub_ls,cond_ls,stim_ls,result_folder,sliding=5)


27
9001 cortisol alc
(25, 377, 377)
9001 cortisol tool
(40, 377, 377)
9001 placebo alc
(40, 377, 377)
9001 placebo tool
(40, 377, 377)
9002 cortisol alc
(40, 377, 377)
9002 cortisol tool
(40, 377, 377)
9002 placebo alc
(40, 377, 377)
9002 placebo tool
(40, 377, 377)
9003 cortisol alc
(40, 377, 377)
9003 cortisol tool
(40, 377, 377)
9003 placebo alc
(40, 377, 377)
9003 placebo tool
(40, 377, 377)
9004 cortisol alc
(40, 377, 377)
9004 cortisol tool
(40, 377, 377)
9004 placebo alc
(40, 377, 377)
9004 placebo tool
(40, 377, 377)
9005 cortisol alc
(40, 377, 377)
9005 cortisol tool
(40, 377, 377)
9005 placebo alc
(40, 377, 377)
9005 placebo tool
(40, 377, 377)
9006 cortisol alc
(40, 377, 377)
9006 cortisol tool
(40, 377, 377)
9006 placebo alc
(40, 377, 377)
9006 placebo tool
(40, 377, 377)
9007 cortisol alc
(40, 377, 377)
9007 cortisol tool
(40, 377, 377)
9007 placebo alc
(40, 377, 377)
9007 placebo tool
(40, 377, 377)
9008 cortisol alc
(40, 377, 377)
9008 cortisol tool
(40, 377, 377)
9008 p